In [1]:
import numpy as np
import pandas as pd
import joblib

In [2]:
filename = 'web/static/data/melon_song_v3.csv'
plist_filename1 = 'web/static/data/melon_playlist1.csv'
plist_filename2 = 'web/static/data/melon_playlist2_v2.csv'
# sim_filename = 'web/static/data/melon_cosine_sim.sim'

df = pd.read_csv(filename)
plist1 = pd.read_csv(plist_filename1)
plist2 = pd.read_csv(plist_filename2)
# cosine_sim = joblib.load(sim_filename)

df.shape, plist1.shape, plist2.shape

((6243, 14), (3845, 5), (60264, 5))

In [ ]:
# import pandas as pd
# df = pd.read_csv('data/melon_song_v3.csv')
# df.head()

In [ ]:
plist1.plylstSeq = plist1.plylstSeq.astype(str)
plist2.songId = plist2.songId.astype(str)
df.songId = df.songId.astype(str)
df.fillna('', inplace=True)
df.head(1)

In [4]:
df.isna().sum()

songId        0
title         0
artist        0
genre         0
album         0
lyricist      0
composer      0
date          0
img           0
comment       0
like          0
lyric         0
plylstSeq     0
ly_summary    0
dtype: int64

In [5]:
with open('data/한글불용어.txt') as st:
    lines = st.readlines()
stop_words = [line.split('\t')[0] for line in lines]
stop_words.extend('은 는 를 도 을 며 의 에 게 니 거 로 요 과 래 랑 파 여 에게'.split())

In [6]:
from konlpy.tag import Okt
okt = Okt()

In [7]:
%%time
reviews = []
for review in df.lyric:
    review = review.replace('\n', ' ')
    morphs = okt.morphs(review, stem=True)
    tmp = [word for word in morphs  if word not in stop_words]
    reviews.append(' '.join(tmp))
df['morphs'] = reviews

Wall time: 2min 47s


In [8]:
# df.to_csv('static/data/melon_song_morphs.csv')

In [ ]:
df.head(5)

In [9]:
df['total'] = df.morphs + (' ' + df.title) + (' ' + df.artist) * 2 + (' ' + df.composer) * 2 + (' ' + df.lyricist) * 2 + (' ' + df.genre) * 3
df.total[0]

'햇살 우릴 위해 내리다 바람 서 롤 감싸다 하다 우리 웃음 속 계절 오다 또 가다 바람 흔들리다 머릿결 내다 불어오다 그대 향기 예쁘다 눈 웃음 소리 모두 가 내 이다 이렇다 사랑 이렇다 행복 쉬다 하다 이렇다 웃음 이렇다 축복 내다 쉬다 올리다 없다 눈물 조차 울음 조차 닦다 못 한 나 정말로 울면 내 가 대다 보내다 준 같다 그대 떠나가다 순간 나르다 걱정 하다 무엇 해주다 없다 내 맘 앞 에서 그대 나르다 떠나다 간다 하다 난 대다 보내다 적 없다 여전하다 대다 나르다 살 하다 이율 테 이렇다 사랑 이렇다 행복 쉬다 하다 이렇다 웃음 이렇다 축복 내다 쉬다 올리다 없다 눈물 조차 울음 조차 닦다 못 한 나 정말로 울면 내 가 대다 보내다 준 같다 그대 떠나가다 순간 나르다 걱정 하다 무엇 해주다 없다 내 맘앞 에서 그대 나르다 떠나다 간다 하다 난 대다 보내다 적 없다 기다림 으로 다시 시 작 일 테 얼마나 사랑 하다 얼마나 또 울다 그대 한순간 조차 잊다 말다 거기 떠나가다 곳 에서 날 기억 하다 기다리다 하루 씩 그대 다가가다 나 이다 희재 성시경 성시경 MGR MGR 양재선 양재선 발라드 국내영화 발라드 국내영화 발라드 국내영화'

In [10]:
# df.set_index('songId', inplace=True)
# df.reset_index(inplace=True)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(stop_words='english')
total_tv = tvect.fit_transform(df.total)
total_tv.shape

(6243, 29665)

In [11]:
indices = pd.Series(df.index, index=df.songId)

In [13]:
indices

songId
418168         0
35609035       1
1046278        2
35609034       3
35728845       4
            ... 
33013736    6238
3080890     6239
2981597     6240
34183979    6241
31191637    6242
Length: 6243, dtype: int64

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(total_tv)

In [13]:
def get_recommendation(title, cos_sim):
    index = indices[title]
    sim_scores = pd.Series(cos_sim[index])
    song_indices = sim_scores.sort_values(ascending=False).head(11).tail(10).index
    return df.songId.iloc[song_indices], df.title.iloc[song_indices], df.artist.iloc[song_indices]

In [16]:
# new_cosine_sim = joblib.load('data/melon_cosine_sim.sim')
# get_recommendation('1046278',cosine_sim)
# get_recommendation('1046278',new_cosine_sim)

(3069    3593455
 14      1046284
 2751     523088
 2502     611573
 2619      19175
 108     2070559
 20      2043348
 307     2070560
 3751    2278760
 3058    3585793
 Name: songId, dtype: object,
 3069           서쪽 하늘
 14                인연
 2751              인연
 2502      오직 너뿐인 나를…
 2619              고백
 108     그런 사람 또 없습니다
 20      그런 사람 또 없습니다
 307            듣고있나요
 3751    그런 사람 또 없습니다
 3058         그리워 그리워
 Name: title, dtype: object,
 3069    울랄라세션
 14        이승철
 2751      이승철
 2502      이승철
 2619      이승철
 108       이승철
 20        이승철
 307       이승철
 3751      이승철
 3058       노을
 Name: artist, dtype: object)

In [17]:
# import joblib

# joblib.dump(cosine_sim, 'data/melon_cosine_sim.sim')

['data/melon_cosine_sim.sim']

In [ ]:
# 사용하려면 불러오는 방법
# new_cosine_sim = joblib.load('data/melon_cosine_sim.sim')

In [29]:
songId = df.songId.values[1]

In [65]:
df.drop(labels=['best_per'],axis=1,inplace=True)

In [ ]:
df.drop(labels=['best_per','best_title', 'best_artist', 'best_songId'],axis=1,inplace=True)

In [66]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim_total = linear_kernel(total_tv, total_tv)

best_songs = []
best_per = []
best_title = []
best_artist = []
count = 0
real_count = 0
other_count = 0
for songId in df.songId.values:
    sim_scores = pd.Series(cosine_sim_total[indices[songId]])
    score = sim_scores.sort_values(ascending=False).head(30).tail(29)

    # best_per.append(f'{score.values[0]:.3f}')
    # best_title.append(df.title.iloc[score.index[0]])
    # best_artist.append(df.artist.iloc[score.index[0]])
    # best_songs.append(df.songId.iloc[score.index[0]])

    count += 1
    per_list= []
    ncount = 0
    for index, per in score.items():
        
        
        # if songId == df.songId.iloc[index] or per == 1.0 or per > 0.83:
        if songId == df.songId.iloc[index] :
            continue
        else:
            real_count += 1
            ncount += 1
            # tmp = per.round(3)
            per_list.append(per)
            # best_per.append(f'{per:.3f}')
            # best_title.append(df.title.iloc[index])
            # best_artist.append(df.artist.iloc[index])
            # best_songs.append(df.songId.iloc[index])
            other_count += 1
            if ncount == 3: break
    best_per.append(per_list)
        
print(count, real_count, other_count)

6243 18729 18729


In [96]:
real_count

6218

In [67]:
df['best_per'] = best_per
# # df['best_title'] = best_title
# # df['best_artist'] = best_artist
# # df['best_songId'] = best_songs
# df.best_per = df.best_per.astype(float)

In [ ]:
df.head()

In [ ]:
df[df.best_per > 0.50]

In [79]:
sim_scores = pd.Series(cosine_sim_total[indices['5445245']])
score = sim_scores.sort_values(ascending=False).head(10)
if score.values[1] < 0.9999999999999999999999999999999999999:
    print('ok')

ok


In [20]:
df_per = df[['songId', 'title', 'artist', 'best_per', 'best_title', 'best_artist', 'best_songId']].sort_values('best_per', ascending=False)

In [21]:
df_per.best_per.mean()

0.32811725132148

In [23]:
df_per.std()

C:\Users\sujung\AppData\Local\Temp\ipykernel_21004\116105951.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_per.std()


best_per    0.176555
dtype: float64

In [59]:
df_per

,songId,title,artist,best_per,percil
0,418168,희재,성시경,"[0.9870539281944808, 0.9322564810500571, 0.356...",0.570871
1,35609035,인생은 뷰티풀,김호중,"[0.4497443003387747, 0.38555240416102976, 0.20...",0.280160
2,1046278,서쪽 하늘,이승철,"[0.8955546907408942, 0.4135766388186803, 0.354...",0.450479
3,35609034,슬픈등,김호중,"[0.4497443003387747, 0.3911845257185991, 0.232...",0.306407
4,35728845,고백 (영화 '동감' X 츄 (이달의 소녀)),츄(Chuu),"[0.9453702657137346, 0.9383494331883004, 0.254...",0.524731
...,...,...,...,...,...
6238,33013736,낮잠,케이시 (Kassy),"[0.4323437177961725, 0.4245417960419125, 0.418...",0.420065
6239,3080890,그대 내게 행복을 주는 사람,해바라기,"[0.9172395612135119, 0.8700807841853618, 0.271...",0.514299
6240,2981597,서른 즈음에,김광석,"[0.9251710743533317, 0.45576962295330997, 0.31...",0.436592
6241,34183979,인생 한잔,정은지,"[0.41184214158735755, 0.2812326657717477, 0.26...",0.286951


In [68]:
df_per = df[['songId', 'title', 'artist', 'best_per']]
df_per['percil'] = [np.mean(x) for x in df_per.best_per]

C:\Users\sujung\AppData\Local\Temp\ipykernel_21004\2869858997.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_per['percil'] = [np.mean(x) for x in df_per.best_per]


In [69]:
df_per.percil.mean(), df_per.percil.median(), df_per.percil.std(), df_per.percil.describe()

(0.34513254403598065,
 0.3021355230267113,
 0.16304490539665006,
 count    6243.000000
 mean        0.345133
 std         0.163045
 min         0.039842
 25%         0.231017
 50%         0.302136
 75%         0.430867
 max         0.971573
 Name: percil, dtype: float64)